In [1]:
!nvidia-smi

Mon Mar  3 08:16:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40S                    On  |   00000000:47:00.0 Off |                    0 |
| N/A   30C    P8             34W /  350W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip3 install torch torchvision

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.

In [3]:
!pip install salesforce-lavis

  Using cached salesforce_lavis-1.0.2-py3-none-any.whl.metadata (18 kB)
  Using cached contexttimer-0.3.3-py3-none-any.whl
  Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl.metadata (422 bytes)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached fairscale-0.4.4-py3-none-any.whl
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached opencv_python_headless-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached opendatasets-0.1.22-py3-none-any.whl.metadata (9.2 kB)
  Using cached plotly-6.0.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pre_commit-4.1.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached pycocoevalcap-1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pycocotools-2.0.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using 

In [4]:
import os
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torchvision.transforms import Compose
from PIL import Image
# from torch.utils.tensorboard import SummaryWriter
from lavis.models import load_model_and_preprocess

/opt/conda/lib/python3.11/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/opt/conda/lib/python3.11/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [5]:
# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "blip2_feature_extractor"
model_type = "pretrain"  # Use "pretrained" as a starting point
batch_size = 16
num_epochs = 10
lr = 1e-5
# log_dir = "logs/run_test"  # TensorBoard log directory
save_model_path = "blip2_finetuned_v2.pt"

In [6]:
# Load the model and preprocessors
model, vis_processors, txt_processors = load_model_and_preprocess(
    model_name, model_type=model_type, is_eval=False
)
model = model.to(device)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
freeze vision encoder
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/LAVIS/models/BLIP2/blip2_pretrained.pth


In [8]:
import os
import boto3
from botocore.exceptions import NoCredentialsError

In [9]:
object_storage_service_name = "source-images-service"
object_storage_namespace = ".ezdata-system"
resource_type = ".svc"
domain = ".cluster.local"
object_storage_port = "30000"


s3_endpoint_url = f"http://{object_storage_service_name}{object_storage_namespace}{resource_type}{domain}:{object_storage_port}"
print(s3_endpoint_url)


s3_client = boto3.client('s3', endpoint_url=s3_endpoint_url)
s3_resource = boto3.resource('s3', endpoint_url=s3_endpoint_url)

http://source-images-service.ezdata-system.svc.cluster.local:30000


In [28]:
bucket = s3_resource.Bucket('poc-mercedes-gp')

training_list = []
for bucket_object in bucket.objects.all():
    if bucket_object.key.startswith('training/'):
        training_list.append(bucket_object.key)

In [30]:
len(training_list)

827

In [31]:
len(dataset)

1481

In [11]:
import json

bucket_name = "poc-mercedes-gp"
file_key = "training/training_dataset.json"

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
content = response["Body"].read().decode("utf-8")  # Convert to string if it's a text file

dataset = json.loads(content)

In [12]:
dataset[0]

{'s3_key': 'training/03_05Bah_Sunday_Alfa Romeo_013.JPG',
 'text': 'A photo of a floor'}

In [34]:
# Copy all the pictures locally
from pathlib import Path

Path('training_images').mkdir(exist_ok=True)
checked_dataset = []

for item in dataset:
    try:
        s3_client.download_file(bucket_name, item['s3_key'], f'training_images/{item["s3_key"].split("/")[-1]}')
        checked_dataset.append(item)
    except Exception as e:
        print(f'Exception {e} for {item["s3_key"]}')


Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/_RedBull1199.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/RedBullTyresAfterFP10084.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/_V6A3625.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/SNE57046.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/SNE58213.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/SNE58454.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/SNE53636.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/SNE51825.JPG
Exception An error occurred (404) when calling the HeadObject operation: Not Found for training/_J5A0265.JPG

In [43]:
!ls training_images | wc

   1366    2488   49706


In [44]:
!pwd

/mnt/user


In [45]:
# Dataset Class
class CustomDataset(Dataset):
    def __init__(self, data, vis_processors, txt_processors, s3_endpoint_url):
        self.data = data
        self.vis_processors = vis_processors
        self.txt_processors = txt_processors

        self.s3_client = boto3.client('s3', endpoint_url=s3_endpoint_url)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path = os.path.join('training_images', self.data[idx]["s3_key"].split('/')[-1])
        text = self.data[idx]["text"]
        
        # Open the image with PIL
        image = Image.open(path).convert("RGB")
        image = self.vis_processors["train"](image)
        text = self.txt_processors["train"](text)

        return {"image": image, "text": text}

In [46]:
# Split dataset
train_data = checked_dataset[:int(0.8 * len(checked_dataset))]
val_data = checked_dataset[int(0.8 * len(checked_dataset)):]

train_dataset = CustomDataset(train_data, vis_processors, txt_processors, s3_endpoint_url)
val_dataset = CustomDataset(val_data, vis_processors, txt_processors, s3_endpoint_url)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [47]:
# Define contrastive loss
def contrastive_loss(image_features, text_features):
    image_features = F.normalize(image_features, dim=-1).mean(dim=1)
    text_features = F.normalize(text_features, dim=-1)

    logits = torch.mm(image_features, text_features.t())
    labels = torch.arange(len(image_features)).to(logits.device)

    loss_img_to_text = F.cross_entropy(logits, labels)
    loss_text_to_img = F.cross_entropy(logits.t(), labels)

    return (loss_img_to_text + loss_text_to_img) / 2

# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)

In [48]:
import mlflow
import mlflow.pytorch
import torch

def get_token():
    TOKEN_PATH = '/etc/secrets/ezua/.auth_token'
    with open(TOKEN_PATH, 'r') as f:
        auth_token = f.read()

    os.environ['MLFLOW_TRACKING_TOKEN'] = auth_token
    os.environ['AUTH_TOKEN'] = auth_token
    return auth_token

In [108]:
token = get_token()

os.environ['MLFLOW_TRACKING_URI'] = 'http://mlflow.mlflow.svc.cluster.local:5000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://local-s3-service.ezdata-system.svc.cluster.local:30000'
os.environ['MLFLOW_S3_IGNORE_TLS'] = 'true'
os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = 'true'
os.environ['MLFLOW_TRACKING_TOKEN'] = token
os.environ['AUTH_TOKEN'] = token

mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

In [109]:
# Set MLflow experiment
mlflow.set_experiment("BLIP 2 F1 finetuning")

<Experiment: artifact_location='s3://mlflow.fr2pcai169/6', creation_time=1741092490228, experiment_id='6', last_update_time=1741092490228, lifecycle_stage='active', name='BLIP 2 F1 finetuning', tags={}>

In [110]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)

In [ ]:
%update_token

import threading
import time

def refresh_token():
    secret_file_path = "/etc/secrets/ezua/.auth_token"

    with open(secret_file_path, "r") as file:
    	token = file.read().strip()
    os.environ['MLFLOW_TRACKING_TOKEN']=token
    print('token refreshed')

def start_timer(interval=900):  # 900 seconds = 15 minutes
    def loop():
        while True:
            refresh_token()
            time.sleep(interval)

    thread = threading.Thread(target=loop, daemon=True)
    thread.start()

start_timer()

# Training Loop
print('Starting training')
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    with mlflow.start_run(nested=True):
        mlflow.log_param("epoch", epoch)
        mlflow.log_param("learning_rate", optimizer.param_groups[0]["lr"])
        mlflow.log_param("batch_size", train_loader.batch_size)

        for batch_idx, batch in enumerate(train_loader):
            
            images = batch["image"].to(device)
            texts = batch["text"]
            texts_tokenized = model.tokenizer(
                    texts,
                    padding="max_length",
                    truncation=True,
                    max_length=35,
                    return_tensors="pt",
                ).to(model.device)
    
            # Forward pass
            image_patch_features, image_cls_features = model.forward_image(images)
            text_features = model.forward_text(texts_tokenized)
    
            # Compute loss
            loss = contrastive_loss(image_patch_features, text_features)
    
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            train_loss += loss.item()
    
            # Log batch loss to MLflow
            mlflow.log_metric("Loss/Train", loss.item(), step=epoch * len(train_loader) + batch_idx)
    
        # Evaluate on validation set
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                images = batch["image"].to(device)
                texts = batch["text"]
    
                texts_tokenized = model.tokenizer(
                        texts,
                        padding="max_length",
                        truncation=True,
                        max_length=35,
                        return_tensors="pt",
                    ).to(model.device)
    
                image_patch_features, image_cls_features = model.forward_image(images)
                text_features = model.forward_text(texts_tokenized)
    
                loss = contrastive_loss(image_patch_features, text_features)
                val_loss += loss.item()
    
        # Average losses
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        # Log epoch losses to MLflow
        mlflow.log_metric("Loss/Epoch_Train", train_loss, step=epoch)
        mlflow.log_metric("Loss/Epoch_Val", val_loss, step=epoch)
    
        # Save model checkpoint to MLflow
        checkpoint_path = f"model_epoch_{epoch}.pt"
        torch.save(model.state_dict(), checkpoint_path)
        mlflow.pytorch.log_model(model, artifact_path=f"model_epoch_{epoch}")

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Token successfully refreshed.
Starting training
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed


2025/03/04 17:18:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


token refreshed


2025/03/04 17:20:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run crawling-owl-227 at: http://mlflow.mlflow.svc.cluster.local:5000/#/experiments/6/runs/d690584f793e493fbc4ca35cfbe59a78.
2025/03/04 17:20:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow.mlflow.svc.cluster.local:5000/#/experiments/6.


Epoch 1/10, Train Loss: 2.1145, Val Loss: 2.0954
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
token refreshed
